In [1]:
import mturk
import random
from datetime import datetime
import json
from pprint import pprint
import copy
import numpy as np
from collections import OrderedDict
import pandas as pd
import pymongo
import botocore
import uuid
import pdb
import seaborn as sns
import matplotlib.pyplot as plt
#import pingouin as pg
import krippendorff_alpha as ka

# Prep

In [2]:
data_folder = '../data/campaign/'
config_folder = '../config/campaign/'

with open('../data/language_tests.json','r',encoding='utf-8') as f:
    language_tests = json.load(f)
with open('../config/mongodb_credentials.json','r') as f:
    mongodb_credentials = json.load(f)
    #mongodb_credentials["connection_string"]

In [3]:
""" Connect to MTurk and to the Mongodb database. Set the boolean below to TRUE to use the marketplace and to FALSE to use the sandbox (testing the HITs)"""
create_hits_in_production = False
is_pilot = False

db_client = pymongo.MongoClient(mongodb_credentials["connection_string"])
db = db_client['textual_entailment']

collection_name = 'hit_results' + ('_sandbox' if not create_hits_in_production else '') + ('_pilots' if is_pilot else '')

hit_result_collection = db[collection_name]
hit_result_collection

Collection(Database(MongoClient(host=['cluster0-shard-00-01.hjstc.mongodb.net:27017', 'cluster0-shard-00-00.hjstc.mongodb.net:27017', 'cluster0-shard-00-02.hjstc.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='Cluster0-shard-0', ssl=True), 'textual_entailment'), 'hit_results_sandbox')

In [4]:
from importlib import reload
reload(mturk)
mt = mturk.MTurk()
mt.launch_client(create_hits_in_production)
collection_name

10000.00


'hit_results_sandbox'

In [5]:
""" Ban the spammers! """
to_ban = False
if to_ban:
    with open('./config/banlist.json','r') as f:
        banlist = json.load(f)
    for w in banlist:
        try:
            response = mt.client.create_worker_block(
                WorkerId=w,
                Reason='You are copy and pasting text'
            )
            assert(response['ResponseMetadata']['HTTPStatusCode'] == 200)
        except botocore.exceptions.ClientError as e:
            continue

# Exec

In [6]:
""" Create the tasks by populating the HTML templates using the config file """

task_types = ['single','multiple']
task_type = task_types[0]

with open(config_folder + 'task_config_{}.json'.format(task_type),'r') as f:
    task_content = json.load(f)

TaskAttributes = task_content['task_attributes']

with open(task_content['html_layout'], 'r', encoding='utf-8') as f:
    html_layout = f.read()
    

html_layout = html_layout.replace('${time_thr}$', task_content['time_thr'])
if is_pilot:
    html_layout = html_layout.replace('${pilot_wording}$',
    '''
    <li>
        <span style="color:red">This is a pilot.</span> Helpful and constructive feedback, regardless of
        whether you finished the task, will be compensated. If there are technical errors that prevented
        you from finishing, let us know and we will take them into consideration.
    </li>
    '''                              
    )
else:
    html_layout = html_layout.replace('${pilot_wording}$','')
    
with open(data_folder + task_content['tasks'],'r') as f:
    taskSets = json.load(f)
    
# If you're only testing, just pick one hit and run it once, with no qualification barriers
if not create_hits_in_production:
    TaskAttributes.pop('QualificationRequirements')
    TaskAttributes['MaxAssignments'] = 1 
    random.seed(42)
    #taskSets = random.sample(taskSets,1)
        
taskSets_dict = {
    'html_layout' : html_layout,
    'taskSets' : taskSets,
    'TaskAttributes' : TaskAttributes,
    'task_content': task_content
}
      
print(f'Generated {len(taskSets)} tasks with the following configs:')

pprint(TaskAttributes,indent=1) #verify the properties before running the HITs

Generated 500 tasks with the following configs:
{'AssignmentDurationInSeconds': 10800,
 'Description': 'Help us by fact-verifying an affirmation. You should have '
                'reading proficiency in English.',
 'Keywords': 'English, Reading, Fact-verification',
 'LifetimeInSeconds': 604800,
 'MaxAssignments': 1,
 'Reward': '0.5',
 'Title': 'Verifying an affirmation with a single evidence.'}


In [7]:
""" See how many HITs this will generate, already multiplied by the expected number of assignments.
Multiply the resulting number by the payment to see how much money this batch will consume. """
    
l=[]
done_count=0
target_assignments = TaskAttributes['MaxAssignments']
for taskSet in taskSets:
    TaskAttributes_hit = copy.deepcopy(TaskAttributes)
    TaskAttributes_hit['MaxAssignments'] = target_assignments -\
        sum([hit['hit']['NumberOfAssignmentsCompleted'] for hit in hit_result_collection.find({
            'taskSet_id':taskSet['_id'],
            'type': task_content['type'],
        })]) #or sum [hit['hit']['NumberOfAssignmentsCompleted'] for completed results
    l.append(TaskAttributes_hit['MaxAssignments'])
    if TaskAttributes_hit['MaxAssignments'] == 0:
        done_count += 1
print(f'Remaining HITs: {sum(l)}')
print(f'Tasks totally finished: {done_count}')
print(f'Expected cost: ${sum(l)*float(TaskAttributes["Reward"])*1.2}')

Remaining HITs: 499
Tasks totally finished: 1
Expected cost: $299.4


In [ ]:
""" Create the batch of HITs """

results = []
batch_id = str(uuid.uuid4())

start_from = 388

hit_type_id = ''
target_assignments = TaskAttributes['MaxAssignments']
for idx, taskSet in enumerate(taskSets):
    print(f"{idx}: {taskSet['_id']}")
    if idx<start_from:
        continue
    TaskAttributes_hit = copy.deepcopy(TaskAttributes) # Adjust based on how many were already done in other batches
    TaskAttributes_hit['MaxAssignments'] = target_assignments -\
        sum([hit['hit']['NumberOfAssignmentsCompleted'] for hit in hit_result_collection.find({
            'taskSet_id':taskSet['_id'],
            'type': task_content['type']
        })])
    if TaskAttributes_hit['MaxAssignments'] > 0:
        try:
            random.seed(None)
            language_questions = random.sample(language_tests['en'],k=4)
            response = mt.create_hit(
                html_layout.replace('${affirmation_evidence_pairs}$', str(taskSet['taskSet'])).\
                            replace('${attention_questions}$', json.dumps(language_questions)),
                **TaskAttributes_hit
            )

            hit_type_id = response['HIT']['HITTypeId']
            result = {
                '_id': response['HIT']['HITId'],
                'batch_id': batch_id,
                'type': task_content['type'],
                'taskSet': taskSet['taskSet'],
                'attention_test': language_questions,
                'taskSet_id':taskSet['_id'],
                'hit': response['HIT'],
                'timestamp': datetime.now()
            }
            results.append(result)
            hit_result_collection.insert_one(result)
        except botocore.exceptions.ClientError as e:
            print(e.__dict__)
            if e.response['Error']['Code'] == 'RequestError':
                # Not enough funds
                print("Funds ran out! The last hit above was not launched! Please recharge!")
                break
            elif e.response['Error']['Code'] == 'ThrottlingException':    
                pdb.set_trace()              
                print("Turn off the database updater!")
                continue
            else:
                pdb.set_trace()            
                raise
        except Exception as e:
            pdb.set_trace()            
            raise
        #except:
        #    pdb.set_trace()            
        #    raise

# For you to go to the HITs you just created and test them
print('Launched tasks')
if not create_hits_in_production:
    print('You can view the HITs here:')
    print(mt.mturk_environment['preview']+"?groupId={}".format(hit_type_id))
else:
    print('Launched! Good Luck!')
    
print('Batch ID is',batch_id)

0: 6e5c60ff-475f-42f9-a6ae-f6334e435f3a
1: 6cd38aa5-a872-45a4-ba78-e832fbcaea2e
2: bbb36b23-2e72-4b15-83d1-9a4e95583820
3: 5ae72c04-e354-4beb-b42f-0bdd6d153620
4: 177f88a1-2b1a-4373-8b21-8866e9db12a1
5: 826a3c84-71a4-4664-85e5-1dcf7145c285
6: 7392f8a7-4b83-40f3-b405-afad32faffac
7: ae59ea09-9637-45d3-af00-eaf6c2fd2ee1
8: ea5b258a-dfcc-4567-8fd7-e2a4cdf80178
9: d50c9568-4ef9-439c-a619-0d29be40e173
10: 02ef5848-0c81-4661-b1a0-cd2bf96cb4ce
11: 34e569ee-9f61-4fb3-9347-642d418e5c8e
12: 03e97986-ce5f-43bc-81a4-2212954a2440
13: d61e000f-7ed9-476a-b617-7fb71fb90c97
14: e3df61cf-de1b-4493-b829-5befa3eed70c
15: a40525ac-748f-47aa-a7aa-01281de87703
16: 447582a4-e8b2-467e-9d03-38b7e1f240ea
17: a993c6d9-1e5f-4f56-84c1-76d617f7a1aa
18: 94fd7f04-3d84-4fbe-b9fc-c997f8d39214
19: ce3853d7-439a-461b-bce8-a68ade951692
20: ff4b9a11-0e02-463b-8494-c5312ca7b74c
21: 25773c8d-d1ed-4acd-a445-a6c9cd7a2cf6
22: 265de7dc-3f9f-44fb-b2b3-d482c2ff5477
23: 0be9b6fd-afcf-4e4a-a022-2e0a9e6bac46
24: 771c9b4d-d25c-4848-8c2

389: 7fc3feda-ec91-42df-a81b-3c14a30ed60e
390: 6edfcaeb-7764-4248-937c-ee66c602c13a
391: 548209f4-e81d-435e-9410-de509d05fafe
392: a02e15df-7383-43d8-a07c-3b443c738330
393: ca02b6b6-b8bc-44a0-897c-e86926e42e7c
394: eebffd6d-e2d4-4c03-91e4-bea1b304cb13
395: d047f785-a50c-43ba-8b3a-f9ba47d363d4
396: 5c15c0c6-5041-42a6-a880-fc8d6ab1fbde
397: adafdaac-c340-40b8-9cd3-5ac52d9f1436
398: bef5b541-d262-4937-b900-61a9c7ee658f
399: 504be20c-0db8-42b0-b65e-b9fbffb8865b
400: 609c98cb-4b26-4449-b3de-11f200550022
401: a540173b-c8b7-4603-8819-b2c71a22a610
402: efa3866f-2fae-46a2-b02a-7a9a574b9eed
403: 96ed51b7-df5c-4125-9dc7-28d0761a09d5
404: 0ba76d54-256c-4b04-9f4c-32533f0cdc06
405: a6bfe919-3cb8-4121-b961-2c4d73e46212
406: 7f25c46c-aab4-4158-b795-b9a026ce2b85
407: 16ac3841-ee51-4cbf-8e1b-c6857b64c061
408: e8b5c449-88c0-4b17-9f55-b3faf201630a
409: 2f54d9a9-5d45-486c-a733-46c81ea6cc07
410: 56f29ab9-d0bc-49bb-9d57-4e58a0bb91ae
411: 94fa61b8-4364-4963-8fda-734f7b7bb2bd
412: 151b4e5f-4c03-4a78-838d-42347

In [49]:
# REMOVING FINISHED TASKS FROM THE UPDATE QUEUE

""" If you set 'force' to TRUE, it will abort mission and force an expiry in all HITs and then delete them.
If you only want to remove the completed ones (make them Disposed so the update routine won't loop through tons of
HITs), keep it as FALSE."""
force = False
while True:
    ''' Dispose all hits in the database '''
    query = {'hit.HITStatus': {'$ne': 'Disposed'}}
    #query['type'] = 'relevance'
    if not force:
        query['hit.NumberOfAssignmentsPending'] = 0
        query['hit.NumberOfAssignmentsAvailable'] = 0
    elif force:
        query['hit.NumberOfAssignmentsPending'] = 0
    hit_result_collection_list = list(hit_result_collection.find(query))
    if (not force and len(hit_result_collection_list) == 0) or (force and mt.client.list_hits()['NumResults']==0):
    #if len(list(hit_result_collection.find({'hit.HITStatus': {'$not': {'$eq': 'Disposed'}}, 'type':'relevance'}))) == 0:
        print('Finished')
        break
    for hit in hit_result_collection_list:
        try:
            #print(f'Trying to remove {hit["_id"]}')
            mt.client.delete_hit(HITId = hit['_id'])
            print('Removed',hit['_id'])
        except Exception as e:
            #print('Level 1', hit['_id'], e)
            if force:
                try:
                    mt.client.update_expiration_for_hit(HITId = hit['_id'], ExpireAt=datetime(2017, 1, 1))
                    mt.client.delete_hit(HITId = hit['_id'])
                    print('Removed',hit['_id'])
                except Exception as e:
                    #print('Level 2', hit['_id'],e)
                    pass
            continue

Removed 3KTZHH2OOLRKTN05R70Q2087MD78MK
Finished


# MISC

In [36]:
mt.client.get_account_balance()

{'AvailableBalance': '10000.00',
 'ResponseMetadata': {'RequestId': 'cac83b9b-d08b-477e-a444-cb6347002975',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'cac83b9b-d08b-477e-a444-cb6347002975',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '31',
   'date': 'Wed, 06 Oct 2021 16:05:21 GMT'},
  'RetryAttempts': 0}}

In [8]:
#mt.client.send_bonus(
#    WorkerId='-',
#    BonusAmount='-',
#    AssignmentId='-',
#    Reason='feedback'
#)

{'ResponseMetadata': {'RequestId': 'a038b160-0721-4475-bbfd-ee73cf5fe3b9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a038b160-0721-4475-bbfd-ee73cf5fe3b9',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'date': 'Fri, 13 Aug 2021 18:45:26 GMT'},
  'RetryAttempts': 0}}

In [38]:
# CLEANING LEFTOVERS

mt.client.list_hits(MaxResults=100)
#hitid = '3M93N4X8INZCBZ9T28UOVALJBK2SJO'
#mt.client.update_expiration_for_hit(HITId = hitid, ExpireAt=datetime(2018, 1, 1))
#mt.get_hit_answers(HITId = hitid, approve=True)
#mt.client.delete_hit(HITId = hitid)

{'NumResults': 0,
 'HITs': [],
 'ResponseMetadata': {'RequestId': '473b5051-fa30-48cd-8b87-4bd024bfebdf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '473b5051-fa30-48cd-8b87-4bd024bfebdf',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '26',
   'date': 'Wed, 06 Oct 2021 16:07:04 GMT'},
  'RetryAttempts': 0}}